Step 1: Installing Libraries

In [ ]:
pip install pandas numpy scikit-learn streamlit scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1


In [ ]:
pip install textblob

STEP 2: Load & Preprocess Data

In [ ]:
import pandas as pd
import numpy as np

movies = pd.read_csv("/content/movies.csv")
ratings = pd.read_csv("/content/ratings.csv")

data = pd.merge(ratings, movies, on="movieId")

print(data.head())

   userId  movieId  rating  timestamp                               title  \
0       1        1     3.4  964982703                    Toy Story (1995)   
1       1        2     4.9  964982703                      Jumanji (1995)   
2       1        3     2.7  964982703             Grumpier Old Men (1995)   
3       1        4     2.7  964982703            Waiting to Exhale (1995)   
4       1        5     3.3  964982703  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


STEP 3: Collaborative Filtering (User-Item Matrix)

In [ ]:
movie_user_matrix = data.pivot_table(
    index='title',
    columns='userId',
    values='rating'
).fillna(0)

print(movie_user_matrix.shape)

(10, 10)


STEP 4: Cosine Similarity (Collaborative Filtering)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(movie_user_matrix)
similarity_df = pd.DataFrame(similarity,
                             index=movie_user_matrix.index,
                             columns=movie_user_matrix.index)

STEP 5: Recommendation Function

In [ ]:
def recommend_movies(movie_name, num_recommendations=5):
    if movie_name not in similarity_df.index:
        return ["Movie not found in dataset."]

    similar_scores = similarity_df[movie_name].sort_values(ascending=False)
    recommendations = similar_scores.iloc[1:num_recommendations+1].index
    return recommendations.tolist()


In [ ]:
print(recommend_movies("Toy Story (1995)"))

['Grumpier Old Men (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)', 'Waiting to Exhale (1995)', 'Heat (1995)']


STEP 6: TF-IDF on Genres

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

movies['genres'] = movies['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

content_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
content_similarity_df = pd.DataFrame(
    content_similarity,
    index=movies['title'],
    columns=movies['title']
)

STEP 7: Content-Based Recommendation

In [ ]:
def content_recommend(movie_name, num_recommendations=5):
    if movie_name not in content_similarity_df.index:
        return ["Movie not found."]

    sim_scores = content_similarity_df[movie_name].sort_values(ascending=False)
    recommendations = sim_scores.iloc[1:num_recommendations+1].index
    return recommendations.tolist()

In [ ]:
print(content_recommend("Toy Story (1995)"))

['Jumanji (1995)', 'Tom and Huck (1995)', 'Father of the Bride Part II (1995)', 'Sabrina (1995)', 'Grumpier Old Men (1995)']


STEP 10: Builded Streamlit UI Code

In [4]:
import streamlit as st

import pandas as pd

movies = pd.read_csv("/content/movies.csv")

st.title("🎬 Movie Recommendation System")

movie_list = movies['title'].values
selected_movie = st.selectbox("Select a Movie", movie_list)

method = st.radio(
    "Choose Recommendation Type",
    ("Collaborative Filtering", "Content Based")
)

if st.button("Recommend"):
    if method == "Collaborative Filtering":
        recommendations = recommend_movies(selected_movie)
    else:
        recommendations = content_recommend(selected_movie)

    st.subheader("Top 5 Recommendations:")
    for movie in recommendations:
        st.write("👉", movie)

2026-02-25 06:05:55.610 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.753 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2026-02-25 06:05:55.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-25 06:05:55.773 Thread 'MainThread': mi

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

movies = pd.read_csv("/content/movies.csv")

st.title("🎬 Movie Recommendation System")

movie_list = movies['title'].values
selected_movie = st.selectbox("Select a Movie", movie_list)

method = st.radio(
    "Choose Recommendation Type",
    ("Collaborative Filtering", "Content Based")
)

if st.button("Recommend"):
    # Assuming recommend_movies and content_recommend are defined in the Colab notebook
    # and accessible when this script is run.
    if method == "Collaborative Filtering":
        recommendations = recommend_movies(selected_movie)
    else:
        recommendations = content_recommend(selected_movie)

    st.subheader("Top 5 Recommendations:")
    for movie in recommendations:
        st.write("👉", movie)

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.42.50:8501

